<a href="https://colab.research.google.com/github/praveenpoluri/CS5590-0004/blob/master/icp8/L5_Encrypted_Federated_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Section: Securing Federated Learning

- Lesson 1: Trusted Aggregator
- Lesson 2: Intro to Additive Secret Sharing
- Lesson 3: Intro to Fixed Precision Encoding
- Lesson 4: Secret Sharing + Fixed Precision in PySyft
- Final Project: Federated Learning wtih Encrypted Gradient Aggregation

# Lesson: Federated Learning with a Trusted Aggregator

In the last section, we learned how to train a model on a distributed dataset using Federated Learning. In particular, the last project aggregated gradients directly from one data owner to another. 

However, while in some cases it could be ideal to do this, what would be even better is to be able to choose a neutral third party to perform the aggregation.

As it turns out, we can use the same tools we used previously to accomplish this.

# Project: Federated Learning with a Trusted Aggregator

### Step 1: Create Data Owners

In [81]:
!pip install syft

In [82]:
import syft as sy
import torch as th
from torch import nn, optim

hook = sy.TorchHook(th)

W0714 16:23:50.295611 140425218107264 hook.py:98] Torch was already hooked... skipping hooking process


In [83]:
# create a couple workers

bob = sy.VirtualWorker(hook, id="bob")
alice = sy.VirtualWorker(hook, id="alice")
secure_worker = sy.VirtualWorker(hook, id="secure_worker")

# this step is important in the real-world application.
# You need to inform the workers of others existance
# you will probably have an ssh or http worker, not a virtual worker.

bob.add_workers([alice, secure_worker])
alice.add_workers([bob, secure_worker])
secure_worker.add_workers([alice, bob])

W0714 16:23:54.134626 140425218107264 base.py:628] Worker alice already exists. Replacing old worker which could cause                     unexpected behavior
W0714 16:23:54.142057 140425218107264 base.py:628] Worker secure_worker already exists. Replacing old worker which could cause                     unexpected behavior
W0714 16:23:54.145772 140425218107264 base.py:628] Worker bob already exists. Replacing old worker which could cause                     unexpected behavior
W0714 16:23:54.149968 140425218107264 base.py:628] Worker secure_worker already exists. Replacing old worker which could cause                     unexpected behavior
W0714 16:23:54.155031 140425218107264 base.py:628] Worker alice already exists. Replacing old worker which could cause                     unexpected behavior
W0714 16:23:54.158508 140425218107264 base.py:628] Worker bob already exists. Replacing old worker which could cause                     unexpected behavior


<VirtualWorker id:secure_worker #objects:45>

In [0]:
# A Toy Dataset
data = th.tensor([[0,0],[0,1],[1,0],[1,1.]], requires_grad=True)
target = th.tensor([[0],[0],[1],[1.]], requires_grad=True)

# get pointers to training data on each worker by
# sending some training data to bob and alice
bobs_data = data[0:2].send(bob)
bobs_target = target[0:2].send(bob)

alices_data = data[2:].send(alice)
alices_target = target[2:].send(alice)


### Step 2: Create the Model

In [0]:
# Iniitalize A Toy Model
model = nn.Linear(2,1)

### Step 3: Send a Copy of The Model to Alice and Bob

In [0]:
bobs_model = model.copy().send(bob) 
alices_model = model.copy().send(alice) 

bobs_opt = optim.SGD(params=bobs_model.parameters(), lr=0.1)
alices_opt = optim.SGD(params=alices_model.parameters(), lr=0.1)


### Step 4: Train Bob's and Alice's Models (in parallel)

In [92]:
for i in range(10):

    # Train Bob's Model
    bobs_opt.zero_grad()
    bobs_pred = bobs_model(bobs_data)
    bobs_loss = ((bobs_pred - bobs_target)**2).mean()
    bobs_loss.backward()

    bobs_opt.step()
    bobs_loss = bobs_loss.get().data
    print(f'Bob loss {bobs_loss}')

    # Train Alice's Model
    alices_opt.zero_grad()
    alices_pred = alices_model(alices_data)
    alices_loss = ((alices_pred - alices_target)**2).mean()
    alices_loss.backward()

    alices_opt.step()
    alices_loss = alices_loss.get().data
    print(f'alice loss {alices_loss}')


Bob loss 0.05534515529870987
alice loss 0.01747691072523594
Bob loss 0.051183927804231644
alice loss 0.015975693240761757
Bob loss 0.04734087735414505
alice loss 0.014604834839701653
Bob loss 0.043789271265268326
alice loss 0.013352017849683762
Bob loss 0.04050569236278534
alice loss 0.012206793762743473
Bob loss 0.03746919706463814
alice loss 0.011159845627844334
Bob loss 0.03466079756617546
alice loss 0.010202687233686447
Bob loss 0.03206315264105797
alice loss 0.009327633306384087
Bob loss 0.02966032549738884
alice loss 0.008527625352144241
Bob loss 0.027437642216682434
alice loss 0.007796232122927904


### Step 5: Send Both Updated Models to a Secure Worker

In [0]:
alices_model.move(secure_worker) # the move function iterates on all objects in alice and call send on it. NOTEICE: inline function
bobs_model.move(secure_worker)


In [94]:
secure_worker._objects # can you identify those objects in the Secure worker?

{1582595598: Parameter containing:
 tensor([[0.6751, 0.0129]], requires_grad=True),
 2008197476: Parameter containing:
 tensor([[ 0.5692, -0.0800]], requires_grad=True),
 2608838588: Parameter containing:
 tensor([[0.2492, 0.0526]], requires_grad=True),
 4337864218: Parameter containing:
 tensor([0.0418], requires_grad=True),
 6487486470: Parameter containing:
 tensor([0.1919], requires_grad=True),
 7429185308: Parameter containing:
 tensor([[ 0.1746, -0.1485]], requires_grad=True),
 8166943004: Parameter containing:
 tensor([0.1012], requires_grad=True),
 8799551038: Parameter containing:
 tensor([0.6128], requires_grad=True),
 10000900803: Parameter containing:
 tensor([0.0791], requires_grad=True),
 10847129305: Parameter containing:
 tensor([[0.3692, 0.0321]], requires_grad=True),
 14717270572: Parameter containing:
 tensor([[0.5185, 0.0208]], requires_grad=True),
 19401305109: Parameter containing:
 tensor([0.0615], requires_grad=True),
 20609823856: Parameter containing:
 tensor(

### Step 6: Average The Models

In [95]:
# A pytorch trick: to set the weights: you have to either access the data[idx] 
# or wrap the set function with (with torch.no_grad():)
# below we illustrate the first way
# the next cell demonstrates the complete way:

model.weight.data[0].set_(((alices_model.weight.data + bobs_model.weight.data) / 2).get())
model.bias.data[0].set_(((alices_model.bias.data + bobs_model.bias.data) / 2).get())

tensor([0.5018])

### Step 7: The complete solution over multiple training sessions

In [96]:
iterations = 10
worker_iters = 5

for a_iter in range(iterations):

    bobs_model = model.copy().send(bob)
    alices_model = model.copy().send(alice)

    bobs_opt = optim.SGD(params=bobs_model.parameters(), lr=0.1)
    alices_opt = optim.SGD(params=alices_model.parameters(), lr=0.1)

    for wi in range(worker_iters):
        # Train Bob's Model
        bobs_opt.zero_grad()
        bobs_pred = bobs_model(bobs_data)
        bobs_loss = ((bobs_pred - bobs_target) ** 2).sum()
        bobs_loss.backward()

        bobs_opt.step()
        bobs_loss = bobs_loss.get().data

        # Train Alice's Model
        alices_opt.zero_grad()
        alices_pred = alices_model(alices_data)
        alices_loss = ((alices_pred - alices_target) ** 2).sum()
        alices_loss.backward()

        alices_opt.step()
        alices_loss = alices_loss.get().data

    alices_model.move(secure_worker)
    bobs_model.move(secure_worker)

    with th.no_grad():

        model.weight.set_(((alices_model.weight.data + bobs_model.weight.data) / 2).get())
        model.bias.set_(((alices_model.bias.data + bobs_model.bias.data) / 2).get())
    
    print("Bob:" + str(bobs_loss) + " Alice:" + str(alices_loss))

Bob:tensor(0.1277) Alice:tensor(0.0411)
Bob:tensor(0.0728) Alice:tensor(0.0136)
Bob:tensor(0.0436) Alice:tensor(0.0050)
Bob:tensor(0.0272) Alice:tensor(0.0018)
Bob:tensor(0.0176) Alice:tensor(0.0006)
Bob:tensor(0.0118) Alice:tensor(0.0002)
Bob:tensor(0.0081) Alice:tensor(3.5445e-05)
Bob:tensor(0.0057) Alice:tensor(2.5743e-06)
Bob:tensor(0.0041) Alice:tensor(9.4453e-07)
Bob:tensor(0.0030) Alice:tensor(5.8414e-06)


In [0]:
preds = model(data)
loss = ((preds - target) ** 2).mean()

In [97]:
print(preds)
print(target)
print(loss.data)

tensor([[0.1256],
        [0.1026],
        [0.8707],
        [0.8477]], grad_fn=<AddmmBackward>)
tensor([[0.],
        [0.],
        [1.],
        [1.]], requires_grad=True)
AutogradTensor>FixedPrecisionTensor>(Wrapper)>[AdditiveSharingTensor]
	-> (Wrapper)>[PointerTensor | me:83834359868 -> bob:48492103601]
	-> (Wrapper)>[PointerTensor | me:47721487910 -> alice:32984600832]
	-> (Wrapper)>[PointerTensor | me:5922426518 -> ted:43222873002]
	-> (Wrapper)>[PointerTensor | me:99005764682 -> carol:83366780188]
	*crypto provider: crypto_provider*


# Lesson: Intro to Additive Secret Sharing

While being able to have a trusted third party to perform the aggregation is certainly nice, in an ideal setting we wouldn't have to trust anyone at all. This is where Cryptography can provide an interesting alterantive. 

Specifically, we're going to be looking at a simple protocol for Secure Multi-Party Computation called Additive Secret Sharing. This protocol will allow multiple parties (of size 3 or more) to aggregate their gradients without the use of a trusted 3rd party to perform the aggregation. In other words, we can add 3 numbers together from 3 different people without anyone ever learning the inputs of any other actors.

Let's start by considering the number 5, which we'll put into a varible x

In [0]:
x = 5

Let's say we wanted to SHARE the ownership of this number between two people, Alice and Bob. We could split this number into two shares, 2, and 3, and give one to Alice and one to Bob

In [99]:
bob_x_share = 2
alice_x_share = 3

decrypted_x = bob_x_share + alice_x_share
decrypted_x

5

Note that neither Bob nor Alice know the value of x. They only know the value of their own SHARE of x. Thus, the true value of X is hidden (i.e., encrypted). 

The truly amazing thing, however, is that Alice and Bob can still compute using this value! They can perform arithmetic over the hidden value! Let's say Bob and Alice wanted to multiply this value by 2! If each of them multiplied their respective share by 2, then the hidden number between them is also multiplied! Check it out!

In [100]:
bob_x_share *= 2
alice_x_share *= 2

decrypted_x = bob_x_share + alice_x_share
decrypted_x

10

This even works for addition between two shared values!!

In [101]:
# encrypted "5"
bob_x_share = 2
alice_x_share = 3

# encrypted "7"
bob_y_share = 5
alice_y_share = 2

# encrypted 5 + 7
bob_z_share = bob_x_share + bob_y_share
alice_z_share = alice_x_share + alice_y_share

decrypted_z = bob_z_share + alice_z_share
decrypted_z

12

As you can see, we just added two numbers together while they were still encrypted ( you are the dealer, thus still can see --and know everything-- but Alice and Bob did not really know anything about the overall deal --the sum of the numbers; the secrete--)

In this case, however, to keep the secrete is not about the relaiability / integrity of the person who holds it, rather the fact that those shareholders do not know each other!!!

One small tweak - notice that since all our numbers are positive, it's possible for each share to reveal a little bit of information about the hidden value, namely, it's always greater than the share. Thus, if Bob has a share "3" then he knows that the encrypted value is at least 3.

This would be quite bad, but can be solved through a simple fix. Decryption happens by summing all the shares together MODULUS some constant.

In [102]:
x = 5

Q = 9923 # the little fix; a very large prime number

bob_x_share = 1525 # give bob a random number.. try with negatives

alice_x_share = Q - bob_x_share + x

print(f'Bob share= {bob_x_share}')
print(f'Alice share= {alice_x_share}')
print(f'Add both shares= {bob_x_share + alice_x_share}... Wrong secrete')

Bob share= 1525
Alice share= 8403
Add both shares= 9928... Wrong secrete


In [103]:
# Decryption happens by summing all the shares together MODULUS some constant

(bob_x_share + alice_x_share) % Q

5

So now, as you can see, both shares are wildly larger than the number being shared, meaning that individual shares no longer leak this inforation. However, all the properties we discussed earlier still hold! (addition, encryption, decryption, etc.)

# Project: Build Methods for Encrypt, Decrypt, and Add 

In this project, you must take the lessons we learned in the last section and write general methods for encrypt, decrypt, and add. Store shares for a variable in a tuple like so:  `x_share = (2,5,7)`

Even though normally those shares would be distributed among several workers, you can store them in ordered tuples like this for now.

In [0]:
import random

Q = 23740629843760239486723

# accepts a secrete and shares it among n_share, returns a tuple with n_share shares
def encrypt(x, n_share=3):
    
    shares = list()
    
    for i in range(n_share-1):
        shares.append(random.randint(0,Q))
        
    shares.append(Q - (sum(shares) % Q) + x)
    
    return tuple(shares)

  # accepts a tuple of shares and return the decrypted values
def decrypt(shares):
    return sum(shares) % Q

In [105]:
shares = encrypt(x=7, n_share=10)
shares

(11846388035402961246969,
 12974871210695313376904,
 10155052958965412165918,
 7926239489144134908440,
 14962645694711250721143,
 5524858162704518160081,
 2025871261317281887348,
 14510315535076991266375,
 17340361715318757431266,
 21436545155464576269178)

In [106]:
decrypt(shares)

7

In [0]:
def add(a, b):
    c = list()
    assert(len(a) == len(b))
    for i in range(len(a)):
        c.append((a[i] + b[i]) % Q)
    return tuple(c)

In [108]:
# run this block of code multiple times, what do you notice baout x and y?

x = encrypt(5)
print(x)
y = encrypt(7)
print(y)

z = add(x,y)
decrypt(z)

(20838997114397793423993, 22650060201527303191086, 3992202371595382358372)
(1226278995304026279156, 19493661577733317030758, 3020689270722896176816)


12

# Lesson: Intro to Fixed Precision Encoding

As you may remember, our goal is to aggregate gradients using this new Secret Sharing technique. However, the protocol we've just explored in the last section uses positive integers. However, our neural network weights are NOT integers. Instead, our weights are decimals (floating point numbers).

Not a huge deal! We just need to use a fixed precision encoding, which lets us do computation over decimal numbers using integers!

In [0]:
BASE= 10  # integer numbers (normal numbers)
PRECISION= 3  # how many porecisions after the decimal point to encode
Q = 23740629843760239486723 

In [0]:
def encode(x):
    return int((x * (BASE ** PRECISION)) % Q)

def decode(x):
    return (x if x <= Q/2 else x - Q) / BASE**PRECISION

In [111]:
encode(-0.600)

23740629843760239345664

In [112]:
decode(9323)

9.323

In [113]:
# run this cell multiple times and notice the changes

x = encrypt(encode(5.5))
y = encrypt(encode(5.5))
z = add(x,y)


print(f'Value of x = {x}')
print(f'Value of y = {y}')
print(f'Value of sum = {z}')
print(f'decoded sum = {decode(decrypt(z))}')

Value of x = (6621519947868350902979, 8025422117786324907959, 9093687778105563681285)
Value of y = (10686602354860089354438, 19719519396308305219380, 17075137936352084405128)
Value of sum = (17308122302728440257417, 4004311670334390640616, 2428195870697408599690)
decoded sum = 11.0


# Lesson: Secret Sharing + Fixed Precision in PySyft

While writing things from scratch is certainly educational, PySyft makes a great deal of this much easier for us through its abstractions.

In [114]:
# Its a good idea to restrat the environment at this point

import syft as sy
import torch as th
from torch import nn, optim

hook = sy.TorchHook(th)

# create a couple workers

bob = sy.VirtualWorker(hook, id="bob")
alice = sy.VirtualWorker(hook, id="alice")
secure_worker = sy.VirtualWorker(hook, id="secure_worker")


bob.add_workers([alice, secure_worker])
alice.add_workers([bob, secure_worker])
secure_worker.add_workers([alice, bob])

W0714 16:26:49.426381 140425218107264 hook.py:98] Torch was already hooked... skipping hooking process
W0714 16:26:49.429251 140425218107264 base.py:628] Worker alice already exists. Replacing old worker which could cause                     unexpected behavior
W0714 16:26:49.430769 140425218107264 base.py:628] Worker secure_worker already exists. Replacing old worker which could cause                     unexpected behavior
W0714 16:26:49.433279 140425218107264 base.py:628] Worker bob already exists. Replacing old worker which could cause                     unexpected behavior
W0714 16:26:49.434546 140425218107264 base.py:628] Worker secure_worker already exists. Replacing old worker which could cause                     unexpected behavior
W0714 16:26:49.437557 140425218107264 base.py:628] Worker alice already exists. Replacing old worker which could cause                     unexpected behavior
W0714 16:26:49.439105 140425218107264 base.py:628] Worker bob already exists. Replacing 

<VirtualWorker id:secure_worker #objects:89>

In [0]:
x = th.tensor([1,2,3,4,5]) ##### the following steps work for integer values only

### Secret Sharing Using PySyft

We can share using the simple` .share()` method!

In [116]:
x = x.share(bob, alice, secure_worker) # implemets additive secrete shares
x # it's a pointer to three secrete shares

(Wrapper)>[AdditiveSharingTensor]
	-> (Wrapper)>[PointerTensor | me:31707255745 -> bob:24270442513]
	-> (Wrapper)>[PointerTensor | me:69275446318 -> alice:95854070887]
	-> (Wrapper)>[PointerTensor | me:71560170694 -> secure_worker:62117102884]
	*crypto provider: me*

In [117]:
bob._objects # take a look at one of the shares.. Large random numbers --encrypted

{4164730689: tensor([[0., 0.],
         [0., 1.]], requires_grad=True), 4556527595: tensor([[ 0.0511],
         [-0.0198]], grad_fn=<AddmmBackward>), 20126619949: tensor([[1830708002507383895, 2263355130074537947]]), 21454551104: tensor([[4138977751660586335, 4129090394376407639],
         [4130005867379347301, 4129428666750119995],
         [4127893465457949495, 4125843393955993374],
         [4139638634608415281, 4139151011653310749],
         [4136304007953457938, 4127915038176218860],
         [4125936806126740287, 4130400670610510207],
         [4123456255369926993, 4132414902996903383],
         [4138374107712638589, 4122947590620096571]]), 22229319681: tensor([[1167261756756035735],
         [1167261756756035735],
         [1167261756756035735],
         [1167261756756035735],
         [1167261756756035735],
         [1167261756756035735],
         [1167261756756035735],
         [1167261756756035735]]), 24270442513: tensor([3018033052193549522, 4238951692305445015, 137801749689

and as you can see, Bob now has one of the shares of x! Furthermore, we can still call addition in this state, and PySyft will automatically perform the remote execution for us!

In [118]:
y = x + x
y

(Wrapper)>[AdditiveSharingTensor]
	-> (Wrapper)>[PointerTensor | me:3163214376 -> bob:99016654489]
	-> (Wrapper)>[PointerTensor | me:73455844056 -> alice:46104448810]
	-> (Wrapper)>[PointerTensor | me:93497503933 -> secure_worker:45364164014]
	*crypto provider: me*

In [119]:
bob._objects # bob has another tensor.. why?

{4164730689: tensor([[0., 0.],
         [0., 1.]], requires_grad=True), 4556527595: tensor([[ 0.0511],
         [-0.0198]], grad_fn=<AddmmBackward>), 20126619949: tensor([[1830708002507383895, 2263355130074537947]]), 21454551104: tensor([[4138977751660586335, 4129090394376407639],
         [4130005867379347301, 4129428666750119995],
         [4127893465457949495, 4125843393955993374],
         [4139638634608415281, 4139151011653310749],
         [4136304007953457938, 4127915038176218860],
         [4125936806126740287, 4130400670610510207],
         [4123456255369926993, 4132414902996903383],
         [4138374107712638589, 4122947590620096571]]), 22229319681: tensor([[1167261756756035735],
         [1167261756756035735],
         [1167261756756035735],
         [1167261756756035735],
         [1167261756756035735],
         [1167261756756035735],
         [1167261756756035735],
         [1167261756756035735]]), 24270442513: tensor([3018033052193549522, 4238951692305445015, 137801749689

In [120]:
y.get() # notice that we have doubled the values on each worker!

tensor([ 2,  4,  6,  8, 10])


### Fixed Precision using PySyft

We can also convert a tensor to fixed precision using `.fix_precision()`
This is important when doing FL since the values you are sharing are decimal numbers.

**Q: what values are we sharing? what are the shared secretes?**

In [0]:
x = th.tensor([0.1, 0.2, 0.3, 0.4]) # a decimal tensor!

In [122]:
x

tensor([0.1000, 0.2000, 0.3000, 0.4000])

In [123]:
x = x.fix_prec() # call this function to encode the decimal tensor using fixed precision
x # notice that this is a tensor chain <-------

(Wrapper)>FixedPrecisionTensor>tensor([100, 200, 300, 400])

In [124]:
print(f'Type of x {type(x)}')
print(f'Type of x.child {type(x.child)}')
print(f'Type of x.child.child {type(x.child.child)}') # <-- this is how you get the data from a fixed precision tensor

Type of x <class 'syft.frameworks.torch.tensors.interpreters.native.Tensor'>
Type of x.child <class 'syft.frameworks.torch.tensors.interpreters.precision.FixedPrecisionTensor'>
Type of x.child.child <class 'syft.frameworks.torch.tensors.interpreters.native.Tensor'>


In [125]:
# to decode the previous tensor
print(x.float_prec())

tensor([0.1000, 0.2000, 0.3000, 0.4000])


In [126]:
y = x + x
y

(Wrapper)>FixedPrecisionTensor>tensor([200, 400, 600, 800])

In [127]:
y = y.float_prec()
y

tensor([0.2000, 0.4000, 0.6000, 0.8000])

### Shared Fixed Precision

And of course, we can combine the two!

In [0]:
x = th.tensor([0.1, 0.2, 0.3])

In [129]:
x = x.fix_prec().share(bob, alice)
x

(Wrapper)>FixedPrecisionTensor>(Wrapper)>[AdditiveSharingTensor]
	-> (Wrapper)>[PointerTensor | me:99930901324 -> bob:74491913464]
	-> (Wrapper)>[PointerTensor | me:23951281280 -> alice:13683868331]
	*crypto provider: me*

In [130]:
bob._objects

{4164730689: tensor([[0., 0.],
         [0., 1.]], requires_grad=True), 4556527595: tensor([[ 0.0511],
         [-0.0198]], grad_fn=<AddmmBackward>), 20126619949: tensor([[1830708002507383895, 2263355130074537947]]), 21454551104: tensor([[4138977751660586335, 4129090394376407639],
         [4130005867379347301, 4129428666750119995],
         [4127893465457949495, 4125843393955993374],
         [4139638634608415281, 4139151011653310749],
         [4136304007953457938, 4127915038176218860],
         [4125936806126740287, 4130400670610510207],
         [4123456255369926993, 4132414902996903383],
         [4138374107712638589, 4122947590620096571]]), 22229319681: tensor([[1167261756756035735],
         [1167261756756035735],
         [1167261756756035735],
         [1167261756756035735],
         [1167261756756035735],
         [1167261756756035735],
         [1167261756756035735],
         [1167261756756035735]]), 24270442513: tensor([3018033052193549522, 4238951692305445015, 137801749689

In [0]:
y = x + x

In [132]:
y.get().float_prec()

tensor([0.2000, 0.4000, 0.6000])

# Final Project: Federated Learning with Encrypted Gradient Aggregation

Reuse your project from the Secuered Agggregator above to train the same model using the FL appraoch, but this time use the Additive Sharing Encryption so that the participating members have access to their own models only. There is no secure aggregator in this senario. 
Include four members in this project (Alice, Bob, Ted, and Carol). 

Hint: we will *share* the model with the shareholders, do addition on the encrypted models, and then aggregate the final model locally. 

In [0]:
import matplotlib.pyplot as plt
import numpy as np
from torch import nn, optim
from torch.autograd import Variable


def test_network(net, trainloader):

    criterion = nn.MSELoss()
    optimizer = optim.Adam(net.parameters(), lr=0.001)

    dataiter = iter(trainloader)
    images, labels = dataiter.next()

    # Create Variables for the inputs and targets
    inputs = Variable(images)
    targets = Variable(images)

    # Clear the gradients from all Variables
    optimizer.zero_grad()

    # Forward pass, then backward pass, then update weights
    output = net.forward(inputs)
    loss = criterion(output, targets)
    loss.backward()
    optimizer.step()

    return True


def imshow(image, ax=None, title=None, normalize=True):
    """Imshow for Tensor."""
    if ax is None:
        fig, ax = plt.subplots()
    image = image.numpy().transpose((1, 2, 0))

    if normalize:
        mean = np.array([0.485, 0.456, 0.406])
        std = np.array([0.229, 0.224, 0.225])
        image = std * image + mean
        image = np.clip(image, 0, 1)

    ax.imshow(image)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['left'].set_visible(False)
    ax.spines['bottom'].set_visible(False)
    ax.tick_params(axis='both', length=0)
    ax.set_xticklabels('')
    ax.set_yticklabels('')

    return ax


def view_recon(img, recon):
    ''' Function for displaying an image (as a PyTorch Tensor) and its
        reconstruction also a PyTorch Tensor
    '''

    fig, axes = plt.subplots(ncols=2, sharex=True, sharey=True)
    axes[0].imshow(img.numpy().squeeze())
    axes[1].imshow(recon.data.numpy().squeeze())
    for ax in axes:
        ax.axis('off')
        ax.set_adjustable('box-forced')

def view_classify(img, ps, version="MNIST"):
    ''' Function for viewing an image and it's predicted classes.
    '''
    ps = ps.data.numpy().squeeze()

    fig, (ax1, ax2) = plt.subplots(figsize=(6,9), ncols=2)
    ax1.imshow(img.resize_(1, 28, 28).numpy().squeeze())
    ax1.axis('off')
    ax2.barh(np.arange(10), ps)
    ax2.set_aspect(0.1)
    ax2.set_yticks(np.arange(10))
    if version == "MNIST":
        ax2.set_yticklabels(np.arange(10))
    elif version == "Fashion":
        ax2.set_yticklabels(['T-shirt/top',
                            'Trouser',
                            'Pullover',
                            'Dress',
                            'Coat',
                            'Sandal',
                            'Shirt',
                            'Sneaker',
                            'Bag',
                            'Ankle Boot'], size='small');
    ax2.set_title('Class Probability')
    ax2.set_xlim(0, 1.1)

    plt.tight_layout()

In [134]:
!pip install syft

In [135]:
# create cells as needed. Separate your solution into logical cells.
import syft as sy
import torch as th
from torch import nn, optim

hook = sy.TorchHook(th)

W0714 16:28:39.227456 140425218107264 hook.py:98] Torch was already hooked... skipping hooking process


In [0]:
bob = sy.VirtualWorker(hook, id="bob")
alice = sy.VirtualWorker(hook, id="alice")
ted = sy.VirtualWorker(hook, id="ted")
carol = sy.VirtualWorker(hook, id="carol")
crypto_provider = sy.VirtualWorker(id="crypto_provider", hook=hook)

In [137]:
bob.clear_objects()
ted.clear_objects()
alice.clear_objects()
carol.clear_objects()

<VirtualWorker id:carol #objects:0>

In [138]:
# Clear workers if previously holding any values
print(bob._objects)
print(alice._objects)
print(ted._objects)
print(carol._objects)

{}
{}
{}
{}


In [139]:
# A Toy Dataset
data = th.tensor([[0,0],[0,1],[1,0],[1,1.],[0,0],[1,1],[0,1],[1,1.]], requires_grad=True)
target = th.tensor([[0],[0],[1],[1.],[0],[0],[1],[1.]], requires_grad=True)

# get pointers to training data on each worker by
# sending some training data to bob and alice
data = data.fix_precision().share(bob, alice, ted, carol, crypto_provider=crypto_provider,  requires_grad=True)
target = target.fix_precision().share(bob, alice, ted, carol, crypto_provider=crypto_provider,  requires_grad=True)
print(bob._objects)

{15703967801: tensor([[1328031935904318588, 3684552063509957971],
        [ 903783791522674295, 2423581070562498429],
        [  60980386783572727, 4417700897477477905],
        [1497855390513288654, 1368097050863192840],
        [ 838584950272227952, 2891016758980196732],
        [3236489687029233973, 3621684913487369611],
        [2843526574409667546, 1596477951655862390],
        [ 286927295796001055, 2027484301335925692]]), 15550388343: tensor([[3684007204458618157],
        [ 643258595403534827],
        [2988626985222028154],
        [3123876605441354964],
        [3371536714130295103],
        [3500211536003448418],
        [1741286689387080877],
        [3809397601451440112]])}


In [0]:



model = nn.Linear(2,1)

[link text](https://)

In [0]:
model = model.fix_precision().share(bob, alice, ted, carol, crypto_provider=crypto_provider,  requires_grad=True)
opt = optim.SGD(params=model.parameters(), lr=0.1).fix_precision()

In [142]:
print(model.weight)
print(opt)

Parameter>AutogradTensor>FixedPrecisionTensor>(Wrapper)>[AdditiveSharingTensor]
	-> (Wrapper)>[PointerTensor | me:72221954693 -> bob:98936011325]
	-> (Wrapper)>[PointerTensor | me:69834504128 -> alice:80299507996]
	-> (Wrapper)>[PointerTensor | me:18344317846 -> ted:78521713411]
	-> (Wrapper)>[PointerTensor | me:84732041413 -> carol:63877723060]
	*crypto provider: crypto_provider*
SGD (
Parameter Group 0
    dampening: 0
    lr: FixedPrecisionTensor>tensor([100])
    momentum: 0
    nesterov: False
    weight_decay: 0
)


In [143]:
for i in range(10):

    # Train Model'S
    opt.zero_grad()
    pred = model(data)
    loss = ((pred - target)**2).sum()
    loss.backward()
    opt.step()
    print(F'loss {loss.get().float_precision()}')

AttributeError: ignored

In [144]:
model.weight

Parameter containing:
Parameter>AutogradTensor>FixedPrecisionTensor>(Wrapper)>[AdditiveSharingTensor]
	-> (Wrapper)>[PointerTensor | me:72221954693 -> bob:98936011325]
	-> (Wrapper)>[PointerTensor | me:69834504128 -> alice:80299507996]
	-> (Wrapper)>[PointerTensor | me:18344317846 -> ted:78521713411]
	-> (Wrapper)>[PointerTensor | me:84732041413 -> carol:63877723060]
	*crypto provider: crypto_provider*

In [145]:
#aggregated model 
model = model.get()
print(model.float_precision().weight)

Parameter containing:
tensor([[-0.0850, -0.0510]], requires_grad=True)
